In [25]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import matplotlib
import matplotlib.pyplot as plt
import re

In [2]:
urls = ['https://www.wsdot.com/traffic/trafficalerts/printer.aspx?view=MAIN&action=1']

In [23]:
raw_dict = {}

for url in urls:
    print(url)
    r = requests.get(url)
    soup = BeautifulSoup(r.text)

   

https://www.wsdot.com/traffic/trafficalerts/printer.aspx?view=MAIN&action=1
['I 5']
[['Northbound']]
I-5 Northbound - 
Update 12:23 PM : On I-5 northbound just south of NE 130th St (MP 173) there is a collision blocking the right lane.  Fire Assistance, the Incident Response Team, and the State Patrol have arrived on the scene. Last Updated: 6/24/2019 12:23 PM
More InfoAt milepost 173

['SR 500']
[['Westbound']]
SR 500 Westbound - 
Maintenance on SR 500 westbound at milepost 3.8 near Thurston Way has the right lane closed for pavement repairs and crack sealing beginning at 11:50 am on June 24, 2019 until further notice. Use caution in the area. Last Updated: 6/24/2019 11:51 AM
More InfoAt milepost 4

['I 90']
[['Both', 'Directions']]
I-90 Both Directions - 
Construction on I-90 at the Medical Lake/SR 902 intersection in both directions at milepost 272.8 near SR 902 beginning at 10:48 am on June 24, 2019 until further notice. Crews will be constructing three new roundabouts and a parall

In [81]:
raw_list = []

#parsing for WA DOT
for x in soup.find_all("div", class_='printerSummary'):
    
    text = x.get_text()
    raw_road = text.splitlines()[0].replace('-', ' ')

    #raw road and direction
    road_words = raw_road.split()
       
    if len(road_words) > 2:
        direction = road_words[2:][0]
        road = road_words[0] + ' '+ road_words[1]
    
    else:
        direction = road_words[1:][0]
        road = road_words[0] + 'Street Bridge'

    #mile post
    milepost = str(re.findall(r'\d+', text.splitlines()[-1])).strip('[').strip(']')
    
    #incident
    incident = str(text.splitlines()[1:-1]).strip('[').strip(']')

    raw_list.append([road, direction, incident, milepost])


In [89]:
labels = ['Road', 'Direction', 'Incident_Description', 'MileMarker']
WA_df = pd.DataFrame.from_records(raw_list, columns = labels)



In [90]:
WA_df.head()

,Road,Direction,Incident_Description,MileMarker
0,I 5,Northbound,'Update 12:23 PM : On I-5 northbound just sout...,'173'
1,SR 500,Westbound,'Maintenance on SR 500 westbound at milepost 3...,'4'
2,I 90,Both,'Construction on I-90 at the Medical Lake/SR 9...,'273'
3,US 12,Westbound,"'Road work on US 12 westbound, from milepost 4...","'42', '21'"
4,US 97,Both,'Maintenance on US 97 both directions between ...,"'23', '34'"


In [91]:
WA_df.shape

(83, 4)